<h1> INTRODUCTION</h1>
This code runs a Character-Wise-LSTM on Yoruba text. The algorithm will train character by character on the yoruba text and also generate text character by character. I specifically chose Yoruba text because it has special characters in it.
<h6>The model</h6>
The model is based on Andrej Karpathy's <a href= "http://karpathy.github.io/2015/05/21/rnn-effectiveness/">post on RNNs</a> and <a href= "https://github.com/karpathy/char-rnn">implementation in Torch</a>. Below is the general architecture of the character-wise RNN.

<img src="images/charseq.jpeg" width="500">
<h6>The Data</h6>
The training data is an excerpt from yoruba bible. I have written a scrapper to scrap any chapter online in Yoruba. 

<h6> HOW TO GO ABOUT IT</h6>
<ol>
    <li>Get the data</li>
    <li>The algorithm cant understand strings, so we have to convert it to numbers</li>
    <li>Preprocess the data</li>
    Since its a character-wise training, each character will be fed into the algorithm, one by one. To pass each character into the algorithm, we have to do one <a href=''>hot encoding</a>. Meaning, we will pass array of 1s and 0s, where the column of the character being passed is 1 and the other column is 0. This will be done for each letter in the data. Meaning, the total number of columns will be the total number of unique character in the text. For example. The letter a will be passed as
    <p> [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] and the letter be will be passed as </p>
    <p> [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]</p>
    Where each column represents a unique letter in the alphabet
    <li>Since the data is hundreds of thousand character long, we will have to train the algorithm in batches </li>
    <li> Define the LSTM architecture</li>
    <p> The architecture is pretty basic, one character enters the LSTM at a time. To understand how LSTM works, please chck out the 
</ol>

In [1]:
from scrapper import getchap
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
8x50x97

In [14]:
# Step1: Getting the data
book, validationbook = ['gal','heb','rom','rev','mat','mrk','luk','jhn'], ['heb', 'rom']
chapter, validchapter = 6, 9
data, validData= [], []
for eachbook in book:
    for i in range(chapter):
        data.append(getchap(eachbook, str(i+1)))
        
for eachbook in validationbook:
    for i in range(6,validchapter):
        validData.append(getchap(eachbook, str(i+1)))

In [15]:
datas, valdatas = '\n'.join(data), '\n'.join(validData)

In [29]:
#Visualise the data
print(datas[:200])

1 Paulu Aposteli 
1 Paulu, aposteli tí a rán kì í ṣe láti ọ̀dọ̀ ènìyàn wá, tàbí nípa ènìyàn, ṣùgbọ́n nípa Jesu Kristi àti Ọlọ́run Baba, ẹni tí ó jí i dìde kúrò nínú òkú. 
2 Àti gbogbo àwọn arákùnrin t


In [25]:
# Step2: Tokenise the data
uniquedata = tuple(set(datas))
datacode = {alphabet:code for code,alphabet in dict(enumerate(uniquedata)).items()}
encodedData = np.array([datacode[word] for word in datas])
encodedvalData = np.array([datacode[word] for word in valdatas])

In [43]:
#Visualise tokenised data
encodedData[:400]

array([68,  1, 35, 17, 42, 26, 42,  1, 67,  3,  6, 41, 61, 71, 26, 79,  1,
       45, 68,  1, 35, 17, 42, 26, 42, 10,  1, 17,  3,  6, 41, 61, 71, 26,
       79,  1, 61, 43,  1, 17,  1, 90, 49, 92,  1, 33,  5,  1, 43,  1,  4,
       71,  1, 26, 49, 61, 79,  1, 81, 65, 94, 81, 65,  1, 40, 92,  5, 16,
        2, 92,  1, 30, 49, 10,  1, 61,  2, 15, 43,  1, 92, 43,  3, 17,  1,
       40, 92,  5, 16,  2, 92, 10,  1,  4, 93,  9, 15, 81, 39, 92,  1, 92,
       43,  3, 17,  1, 31, 71, 41, 42,  1, 19, 90, 79, 41, 61, 79,  1,  2,
       61, 79,  1, 48, 26, 81, 39, 90, 42, 92,  1, 84, 17, 15, 17, 10,  1,
       95, 92, 79,  1, 61, 43,  1, 62,  1, 75, 43,  1, 79,  1, 94,  5, 94,
       71,  1, 33,  0, 90, 11,  1, 92, 43, 92,  0,  1, 11, 33,  0, 69,  1,
       45, 59,  1, 77, 61, 79,  1,  9, 15,  6,  9, 15,  6,  1,  2, 30, 81,
       92,  1, 17, 90, 49, 33, 93, 92, 90, 79, 92,  1, 61, 43,  1, 62,  1,
       30,  2,  1,  3, 95, 65, 26,  0,  1, 53, 79, 10, 80, 43,  1,  2, 30,
       81, 92,  1,  5, 75

In [95]:
encodedvalData[:400]

array([12,  1, 18, 71, 26, 33, 79, 41, 71, 94, 71, 33, 79,  1, 75, 95, 39,
        1,  2, 26, 93, 87, 49,  2,  1, 45, 68,  1, 66, 43, 61,  6, 90, 43,
        1, 18, 71, 26, 33, 79, 41, 71, 94, 71, 33, 79,  1, 16,  5, 43, 10,
        1, 81, 15, 17,  1, 80, 17, 26, 95, 53, 42, 10,  1,  2, 26, 93, 87,
       49,  2,  1, 48, 26, 81, 39, 90, 42, 92,  1, 48, 65,  9, 49, 28, 11,
        9,  6, 10,  1, 95, 92, 79,  1, 61, 43,  1, 62,  1,  3,  2, 94, 13,
        1, 67, 15, 90, 17, 52, 17, 53, 42,  1, 15, 43,  1, 62,  1, 61, 79,
        1, 73,  1,  3, 17, 94,  2,  1, 15, 81, 65,  1, 26, 49, 61, 79,  1,
       79, 15, 79,  1,  3, 43,  3, 17,  1,  2, 30, 81, 92,  1, 81, 15, 17,
       10,  1, 61, 43,  1, 62,  1, 41,  5,  1, 41,  0, 90, 71,  1, 87,  0,
       92,  1, 42, 92, 10,  1, 45, 59,  1, 95, 92, 79,  1, 61, 43,  1, 67,
       15, 90, 17, 52, 17, 53, 42,  1, 41,  5,  1,  3, 43, 92,  1,  5, 94,
       49, 53, 95, 39, 30,  2, 49,  1,  6, 52, 42, 92,  1,  9, 15,  6,  9,
       15,  6,  1, 87,  0

In [53]:
encoded_ydata = list(encodedData[1:])
encoded_ydata.append(encodedData[0])
validdatalist = list(encodedvalData[1:])
validdatalist.append(encodedvalData[0])

In [62]:
encodedydata= np.array(encoded_ydata)
encodedvaliddata = np.array(validdatalist)

In [99]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(encodedData), torch.from_numpy(encodedydata))
valid_data = TensorDataset(torch.from_numpy(encodedvalData), torch.from_numpy(encodedvaliddata))

# dataloaders
batch_size = 400

# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=False, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=False, batch_size=batch_size)

In [100]:
dataiter= iter(train_loader)
sample_x, sample_y = dataiter.next()

validiter = iter(valid_loader)
valid_x, valid_y = dataiter.next()

In [112]:
# Step 3. Onehot encoding
def oneHotEncode(arr, cols_num):
    """
    This function takes in an pytorch dataloader object and returns a one-hot encoded array with dimensions of array x n_labels.
    E.G if it takes an array of [3,2,1] and n_labels of 8, it returns a 3x3(array_size by cols_num) hot encoded array like so
    [[0 0 1 0 0 0 0 0]
    [0 1 0 0 0 0 0 0]
    [1 0 0 0 0 0 0 0]]
    """
    array = np.array(arr)
    # First, create an array.size by cols_num array of zeros in float
    one_hot = np.zeros((array.size,cols_num), dtype= np.float32)
    
    # Fill a "1" to each row based on the value in array
    one_hot[np.arange(one_hot.shape[0]), array.flatten()] = 1.
    print(one_hot.shape)
    
    # Return back to the original shape
    one_hot = one_hot.reshape((*array.shape, cols_num))
    print(one_hot.shape)
    return one_hot

In [ ]:
train_on_gpu = 

In [114]:
# Step 4 Model Architecture
class yorLSTM(nn.Module):
    def __init__(self, tokens, n_hidden= 256, n_layers=2,drop_prob=0.5,lr= 0.01):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden= n_hidden
        self.lr = lr
            
        #Here, we are creating character dict for the project
        self.chars = tokens
        self.int2chars = dict(enumerate(self.chars))
        self.char2int = {num:alphabet for alphabet,num in self.int2chars.items()}
            
        # Define model layers
        self.lstm = nn.LSTM(len(self.chars), self.n_hidden, self.n_layers,dropout=drop_prob,batch_frst=True)
            
        # Dropout in between layers
        self.dropout =nn.Dropout(drop_prob)
            
        # Connect to a fully connected layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
        
    def forward(self, x, hidden):
        out, hidden = self.lstm(x, hidden)
        
        # Dropout to avoid overfitting
        out= self.dropout(out)
        
        # Reshape for fully connected layer
        out = out.view(-1, self.n_hidden)
        
        # Pass through a fully connected layer
        out = self.fc(out)
        
        #return out, hidden
        return out, hidden
        
    def init_hidden(self, batch_size):
        # Initialize the weight and hidden value
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers,batch_size,self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        
            

In [116]:
# Step5 Train the Model
def train(network,training_data, validation_data, epochs, batch_size,lr, seq_length=10, clip=5, vis=10):
    # Set the RNN network to train
    network.train()
    
    #Set the optimiser and calculate the loss
    optimiser = torch.optim.Adam(net.parameters(), lr= lr)
    criterion = nn.CrossEntropyLoss()
    
    # Run on CUDA if available
    if (train_on_gpu):
        network.cuda()
        
    counter = 0
    # set total number of characters
    n_chars = len(network.chars)
    
    # Train in range of epochs
    for i in range(epochs):
        # initialise the hidden state
        h = network.init_hidden(batch_size)
        for x, y in training_data:
            counter+=1
            
            # One-Hot-Encode the training data
            x = oneHotEncode(x.reshape(8,50), n_chars)
            
            # If on Cuda, cnvert the x and y to cuda
            if (train_on_gpu):
                x, y = x.cuda(), y.cuda()
            
            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])
            
            # Set accumulated gradient to zero
            network.zero_grad()
            
            output, h = network(x, h)
            
            # Calculate the loss and back propagate
            loss = criterion(output, y.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            optimiser.step()
            
            # Calculate validation loss at every 10 iteration
            if counter% vis == 0:
                # Initialise the hidden state
                val_h = network.init_hidden(batch_size)
                validation_losses = []
                
                #set network to evalution
                network.eval()
                
                for x,y in validation_data:
                    x = oneHotEncode(x.reshape(8,50), n_chars)
                    
                    if (train_on_gpu):
                        x,y = x.cuda(), y.cuda()
                    val_h = tuple([each for each in val_h])
                    
                    output, val_h = network(x, val_h)
                    
                    #Calculate the loss
                    loss = criterion(output, y.view(batch_size* seq_length).long())
                    
                    # Append loss to validation losses
                    validation_losses.append(loss)
                    
                    # Set network back to training
                    network.train()
                    
                    print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))